In [1]:
import sys
import os

from copy import deepcopy
import h5py

import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt

# Import torch
import torch
from torch import nn

myhost = os.uname()[1]
sys.path.insert(0, '/home/elott1/code/')
data_dir = "/home/elott1/data/packaged/"
work_dir = '/home/elott1/code/NTdatasets/hartley/'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device0 = torch.device("cpu")

print(f'Running on Computer: [{myhost}]')
print(f'Data directory: {data_dir}')
print(f'Working directory: {work_dir}')

%load_ext autoreload
%autoreload 2

Running on Computer: [PFC]
Data directory: /home/elott1/data/packaged/
Working directory: /home/elott1/code/NTdatasets/hartley/


In [2]:
# NDN tools
import NDNT.utils as utils          # some other utilities\n",
from NDNT.utils import imagesc      # because I'm lazy\n",
from NDNT.utils import ss           # because I'm real lazy\n",
import NDNT.NDNT as NDN
from NDNT.modules.layers import *
from NDNT.networks import *
from NDNT.utils.DanUtils import ss
from NDNT.utils.DanUtils import imagesc

Invoking __init__.py for NDNT.utils


In [3]:
fn0 = 'Jocamo_220628_full_HC_ETCC_nofix_v08'
f = h5py.File(os.path.join(data_dir, fn0+'.mat'), 'r')

In [4]:
from NTdatasets.hartley.hartley import HartleyDataset

expts = [fn0]

data = HartleyDataset(
    filenames=expts,
    datadir=data_dir,
    num_lags=1,
    time_embed=False,
    include_MUs=False,
    preload=True,
    drift_interval=None,
    device=device0,
    eye_config=3,
    maxT=1000
)

Loading data into memory...
Stim: using laminar probe stimulus
  Adding fixation point
  Done
T-range: 0 1000
  Trimming experiment 36720->1000 time points based on eye_config and Tmax
Extending final block at  960 1000


In [5]:
lbfgs_pars = utils.create_optimizer_params(
    optimizer_type='lbfgs',
    tolerance_change=1e-10,
    tolerance_grad=1e-10,
    history_size=10,
    batch_size=4000,
    max_epochs=3,
    max_iter = 2000,
    device = device)

data.add_covariate('metadata', np.array(data.metas)[:1000])
data.metas.shape

(36720, 4)

In [6]:
XTreg0 = 1.0
L1reg0 = 0.0001

glm_all_par = NDNLayer.layer_dict( 
    input_dims=[1,1,1,data.metas.shape[1]], num_filters=data.NC, 
    bias=True, initialize_center = True,
    NLtype='softplus')
glm_all_par['reg_vals'] = {'d2xt': XTreg0, 'l1':L1reg0, 'localx':0.001, 'bcs':{'d2xt':1}  } 

glm_net = FFnetwork.ffnet_dict(xstim_n='metadata', layer_list=[glm_all_par])

In [14]:
torch.cuda.empty_cache()
## Fit all GLMs at once -- example for choice of regularization above
glm_all = NDN.NDN(ffnet_list=[glm_net])
glm_all.fit(data, force_dict_training=True, **lbfgs_pars, verbose=0)
LLs = glm_all.eval_models(data[data.val_inds], null_adjusted=True)
print(LLs)

[-2.39706039e-03 -7.16829300e-03 -8.72731209e-02 -5.32166958e-02
 -3.32930088e-01 -4.88505363e-02 -8.62479210e-04 -1.03735924e-03
 -1.61218643e-03 -4.57360744e-02 -4.16898727e-02  1.13081932e-03
 -1.81570053e-02 -8.71174335e-02 -1.86923742e-01 -4.83293533e-02
  9.62805748e-03 -1.24532700e-01 -1.43420887e+00 -1.74955368e-01
 -7.92806149e-02 -8.35585594e-03 -1.09011889e-01  6.88982010e-03
 -1.78129673e-01 -1.87892914e-02 -9.65437889e-02  6.33955002e-04
 -3.09023857e-02 -1.12953901e-01 -6.74123764e-02 -3.62658501e-03
 -1.90558195e-01  7.40289688e-04 -1.09565258e-02 -5.76720238e-02
 -7.32498169e-02 -1.04814529e-01 -6.63280487e-03 -5.28273582e-02
 -2.21951008e-02             inf -1.16327524e-01 -1.82256699e-02
 -3.91426086e-02 -2.58633614e-01 -3.58042717e-02 -1.45932436e-01
  3.57737541e-02 -1.37591362e-03 -5.54707050e-02  1.25336647e-03
 -1.90668106e-02 -7.60300159e-02 -2.33979225e-02 -3.69838047e+00
 -5.70478439e-02 -3.91149521e-03 -1.09751701e-01  9.02175903e-03
 -1.60217762e-01 -2.09318

/home/elott1/code/NDNT/NDNT.py:732: RuntimeWarning: divide by zero encountered in log
  LLnulls = np.log(rbar)-1


In [10]:
## Optimize d2xt regularization
reglist = [0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30, 100, 300]
LLsR = np.zeros([len(reglist), data.NC])
data.set_cells()
for rr in range(len(reglist)):
    glm_all_par['reg_vals']['d2xt'] = reglist[rr]
    glm_iter = NDN.NDN(ffnet_list=[glm_net])
    glm_iter.fit(data, force_dict_training=True, **lbfgs_pars, verbose=0)
    LLsR[rr, :] = glm_iter.eval_models(data[data.val_inds], null_adjusted=True)
    print( "%7.2f  %9.7f  %9.7f"%(reglist[rr], np.mean(LLsR[rr,:]), LLsR[rr, 1]) )

  Reset cells_out to full dataset (140 cells).
   0.00        inf  -0.0868900
   0.00        inf  -0.0873508
   0.01        inf  -0.0244875
   0.03        inf  -0.0879474
   0.10        inf  -0.0705743
   0.30        inf  -0.0402524
   1.00        inf  -0.0566220
   3.00        inf  -0.1576395
  10.00        inf  -0.0852189
  30.00        inf  -0.1142206
 100.00        inf  -0.1307681
 300.00        inf  -0.1115551


In [11]:
LLs0 = np.max(np.mean(LLsR, axis=1))
LLs0

inf